# PyTorch Vanilla Autoencoder Lab

### Why this notebook
- Provide a clean reference implementation of the baseline autoencoder architecture.
- Show how to interact with the modular `src` package for configuration, training, and inference.
- Offer experiment prompts you can reuse before branching into advanced variants.

### Learning objectives
- Train the fully-connected autoencoder on Fashion-MNIST and inspect convergence.
- Reconstruct held-out samples and visualise reconstruction fidelity.
- Understand where to customise latent size, optimiser, or scheduler settings.

### Prerequisites
- PyTorch 2.x with torchvision installed.
- Familiarity with the project layout described in `README.md`.
- Optional: GPU/MPS for faster experimentation.

### Notebook workflow
1. Import config and utilities from `../src`.
2. Launch `train(CONFIG)` to fit the model and persist artefacts.
3. Load the trained weights and reconstruct sample images.
4. Explore modifications: latent dimension changes, activation swaps, or convolutional replacements.


**Workflow**

1. Import the package and inspect the configuration.
2. Launch training with automatic device detection (MPS → CUDA → CPU).
3. Reconstruct a held-out image to verify the decoder.

In [ ]:
from pathlib import Path
import sys

NOTEBOOK_DIR = Path().resolve()
SRC_DIR = NOTEBOOK_DIR.parent / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

from config import CONFIG  # noqa: E402
from inference import load_model, reconstruct  # noqa: E402
from train import train  # noqa: E402

CONFIG

In [ ]:
metrics = train(CONFIG)
metrics

### Interpret the metrics
- `metrics` contains per-epoch reconstruction loss and PSNR.
- Use the dictionary to plot curves (e.g., with pandas/Matplotlib) and monitor convergence.
- Compare the values against denoising/sparse variants to gauge baseline performance.
- Persist the metrics next to checkpoints when running automated experiments.

In [ ]:
import matplotlib.pyplot as plt
from torchvision import datasets, transforms

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
test_ds = datasets.FashionMNIST(root=str(CONFIG.data_dir), train=False, download=True, transform=transform)
image, _ = test_ds[0]
model = load_model(config=CONFIG)
reconstruction = reconstruct([image], model=model, config=CONFIG)[0]

def to_numpy(tensor):
    return tensor.squeeze().cpu().numpy() * 0.5 + 0.5

fig, axes = plt.subplots(1, 2, figsize=(6, 3))
axes[0].imshow(to_numpy(image), cmap='gray')
axes[0].set_title('Original')
axes[0].axis('off')
axes[1].imshow(to_numpy(reconstruction), cmap='gray')
axes[1].set_title('Reconstruction')
axes[1].axis('off')
plt.tight_layout()

### Next experiments
- Log reconstruction grids for multiple samples to evaluate qualitative performance.
- Replace the MLP with a convolutional architecture by editing `model.py`.
- Use the learned encoder as a feature extractor for downstream classifiers.
- Run the TensorFlow notebook for a cross-framework comparison of training behaviour.